<a href="https://colab.research.google.com/github/jiedali/colab_temp/blob/main/breakout.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# imports
# Reference: https://www.cs.toronto.edu/~vmnih/docs/dqn.pdf
import os
import time
import numpy as np
%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)
import gym
import random
from collections import deque
import matplotlib.pyplot as plt
# choose a GPU card
os.environ['CUDA_VISIBLE_DEVICES']="2"

TensorFlow 1.x selected.
1.15.2


In [2]:
env = gym.make("BreakoutNoFrameskip-v4")
obs = env.reset()
obs.shape

(210, 160, 3)

In [ ]:
### Helper Functions

In [ ]:
def huber_loss(error):
  

In [3]:
def preprocess(image):
    """ prepro 210x160x3 uint8 frame into 6400 (80x80) 2D float array """
    image = image[35:195] # crop
    image = image[::2,::2,0] # downsample by factor of 2
    image[image == 144] = 0 # erase background (background type 1)
    image[image == 109] = 0 # erase background (background type 2)
    image[image != 0] = 1 # everything else just set to 1
    return np.reshape(image.astype(np.float).ravel(), [80,80])

## Define Q network

In [4]:
# This is now exactly the same as Ghani's CNN settings
# need to adjust to 80**)
learning_rate=0.001
state_size=[80,80,1]
action_size=env.action_space.n
n_outputs=2 # we are only using action 2 and 3 (RIGHT and LEFT)
#
input_height=80
input_width=80
input_channels=1

In [30]:
action_size=env.action_space.n

In [32]:
action_size

4

In [41]:
# Define a Q network with input size of [80,80], and an output size of size of action space 2
# Note that the action space is 4 with following meanings:
# ['NOOP', 'FIRE', 'RIGHT', 'LEFT']
# We will only be taking action 2 or 3
class DQN(object):
    def __init__(self, state_size, action_size, learning_rate, name='online_q_network'):
        self.state_size = state_size
        self.action_size = action_size
        self.learning_rate = learning_rate
        self.name =name
        
        with tf.variable_scope(name):
            with tf.name_scope("inputs"):
                # Jieda note: state is preprocessd 80*80*1 array
                self.inputs = tf.placeholder(tf.float32, [None,*state_size], name = "inputs")

            with tf.name_scope("conv1"):
                self.conv1 = tf.layers.conv2d(
                inputs=self.inputs, filters=32, kernel_size=[8, 8], strides=4,
                kernel_initializer=tf.variance_scaling_initializer(scale=2),
                padding="VALID", activation=tf.nn.relu, use_bias=False, name='conv1')
                
                self.conv1_out = tf.nn.relu(self.conv1, name='conv1_out')
             
            with tf.name_scope("conv2"):
                
                self.conv2 = tf.layers.conv2d(
                inputs = self.conv1_out, filters=64,
                kernel_size=[4,4], strides=[2,2],padding="VALID",
                kernel_initializer = tf.variance_scaling_initializer(scale=2),
                activation=tf.nn.relu, use_bias=False, name='conv2')
                
                self.conv2_out = tf.nn.relu(self.conv2, name='conv2_out')
            
            with tf.name_scope("conv3"):
            
                self.conv3 = tf.layers.conv2d(
                inputs = self.conv2_out, filters=64,
                 kernel_size=[3,3], strides=[1,1], padding="VALID",
                 kernel_initializer = tf.variance_scaling_initializer(scale=2),
                 name = "conv3")
                
                self.conv3_out = tf.nn.relu(self.conv3, name='conv3_out')
            
            with tf.name_scope("flatten"):
                self.flatten = tf.contrib.layers.flatten(self.conv3_out)
                

            with tf.name_scope("fc1"):
                self.fc1 = tf.layers.dense(inputs=self.flatten,
                                          units = 512, activation = tf.nn.relu,
                                          kernel_initializer = tf.contrib.layers.xavier_initializer(), name = "fc1")
            
            with tf.name_scope("fc1"):
                self.fc2 = tf.layers.dense(inputs=self.flatten,
                                          units = 512, activation = tf.nn.relu,
                                          kernel_initializer = tf.contrib.layers.xavier_initializer(), name = "fc2")

            with tf.name_scope("outputs"):
                self.outputs = tf.layers.dense(inputs = self.fc2,
                                              units = action_size,
                                              kernel_initializer = tf.contrib.layers.xavier_initializer(),
                                              activation = None)
            # Output is the approximated Action Values Q(s,a), so we don't need any activation 

    def get_outputs(self):
      
        return self.outputs

    def get_weights(self,scope_name):
        # give all the weights of that network
        trainable_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope=scope_name)
        # create a dictionary to contain the values of the network weights
        trainable_vars_by_name = {var.name[len(scope_name):]: var
                                for var in trainable_vars}
        return trainable_vars_by_name  
#             with tf.name_scope("loss"):
#                 self.cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits = self.logits, labels =self.actions)
#                 # Jieda noted: for baseline, we subtract the value_estimate_ from discounted_episode_rewards
#                 self.weighted_negative_likelihoods = tf.multiply(self.cross_entropy, self.discounted_episode_rewards)
#                 self.loss = tf.reduce_mean(self.weighted_negative_likelihoods)

#             with tf.name_scope("train"):
#                 self.optimizer = tf.train.RMSPropOptimizer(self.learning_rate)
#                 self.train_opt = self.optimizer.minimize(self.loss)
tf.reset_default_graph()
X_state=tf.placeholder(tf.float32,shape=[None,input_height,input_width,input_channels])
# initialize the two Q network
online_q = DQN(state_size, action_size, learning_rate, 'online_q_network')
target_q = DQN(state_size, action_size, learning_rate, 'target_q_network')
# get the output and weights from online q network
online_q_values=online_q.get_outputs()
online_q_weights=online_q.get_weights(scope_name='online_q_network')
# # get the output and weights from target q network
target_q_values=target_q.get_outputs()
target_q_weights=target_q.get_weights(scope_name='target_q_network')
# define the operation to copy the online network weights to target_q_network weights
copy_ops = [target_var.assign(online_q_weights[var_name]) for var_name, target_var in target_q_weights.items()]
#
copy_online_to_target = tf.group(*copy_ops)

## define the train operation
with tf.variable_scope("train"):
  # define training operation
  input_action = tf.placeholder(tf.int32, shape=[None])
  y=tf.placeholder(tf.float32, shape=[None, 1])
  # Get the Q values for the input_action
  q_value = tf.reduce_sum(online_q_values*tf.one_hot(input_action,action_size),axis=1,keepdims=True)
  # compute the error between lable y and q_value from online_q_network approximation
  # Note that lable y is computed using the target_q_network
  error=tf.abs(y-q_value)
  clipped_error = tf.clip_by_value(error,0,1)
  linear_error = 2*(error-clipped_error)
  loss = tf.reduce_mean(tf.square(clipped_error)+linear_error)

  # global_step is used to keep track of number of training steps completed
  global_step = tf.Variable(0,trainable=False,name='global_step')
  optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=0.9, beta2=0.999,epsilon=1e-08,use_locking=False,name='Adam')
  training_op = optimizer.minimize(loss, global_step=global_step)

init=tf.global_variables_initializer()
saver=tf.train.Saver()

In [23]:
target_q_weights.items()

dict_items([('/conv1/kernel:0', <tf.Variable 'target_q_network/conv1/kernel:0' shape=(8, 8, 1, 32) dtype=float32_ref>), ('/conv2/kernel:0', <tf.Variable 'target_q_network/conv2/kernel:0' shape=(4, 4, 32, 64) dtype=float32_ref>), ('/conv3/kernel:0', <tf.Variable 'target_q_network/conv3/kernel:0' shape=(3, 3, 64, 64) dtype=float32_ref>), ('/conv3/bias:0', <tf.Variable 'target_q_network/conv3/bias:0' shape=(64,) dtype=float32_ref>), ('/fc1/kernel:0', <tf.Variable 'target_q_network/fc1/kernel:0' shape=(2304, 512) dtype=float32_ref>), ('/fc1/bias:0', <tf.Variable 'target_q_network/fc1/bias:0' shape=(512,) dtype=float32_ref>), ('/fc2/kernel:0', <tf.Variable 'target_q_network/fc2/kernel:0' shape=(2304, 512) dtype=float32_ref>), ('/fc2/bias:0', <tf.Variable 'target_q_network/fc2/bias:0' shape=(512,) dtype=float32_ref>), ('/dense/kernel:0', <tf.Variable 'target_q_network/dense/kernel:0' shape=(512, 2) dtype=float32_ref>), ('/dense/bias:0', <tf.Variable 'target_q_network/dense/bias:0' shape=(2,)

In [13]:
online_q_values

<tf.Tensor 'online_q_network/outputs/dense/BiasAdd:0' shape=(?, 2) dtype=float32>

In [ ]:
## Learning parameters

In [ ]:
learning_rate=0.001
# Use adam optimizer
beta_1=0.9
beta_2=0.999
epsilon=1e-07

In [ ]:
## Replay Buffer

In [ ]:
replay_memory_size=500000
replay_memory = deque([],maxlen=replay_memory_size)

In [ ]:
class ReplayBuffer:
    def __init__(self,maxlen):
        self.maxlen=maxlen
        self.buf = 

    def draw_samples_from_rb(batch_size):

    

In [ ]:
## Epsilon Greedy

In [ ]:
eps_min = 0.1
eps_max = 1.0
eps_decay_steps = 2000000
def epsilon_greedy(q_values, step):
    # Note: we gradually decrease epsilon, we explore more in the beginning, less towards later
    epsilon = max(eps_min, eps_max-(eps_max-eps_min)*step/eps_decay_steps)
    if np.random.rand()<epsilon:
        return np.random.randint(n_outputs)
    else:
        return np.argmax(q_values)

In [ ]:
## Training parameters

In [ ]:
# This is the number of training steps 
n_steps = 4000000
# In the very beginning, First have 10000 samples in the replay buffer
training_start=10000
# Before each training step, we run 4 episodes and add those samples to replay buffer
training_interval=4
# 
save_steps=1000
copy_steps = 10000 # copy the online network parameters to target network every 10000 steps
discount_rate=0.99
skip_start=90
batch_size=50
iteration=0
done=True

In [ ]:
with tf.Session as sess:
  if os.path.isfile(checkpoint_path+".index"):
    saver.restore(sess, checkpoint_path)
  else:
    init.run()
    # to make sure they have the same values to begin with
    # otherwise, different seeds will result in different initialized network weights
    copy_online_to_target.run()
  # start training
  # Training loop : 
  # (1) collect M data points, add to replay buffer (play 4 episodes of game, add those to replay buffer)
  # (2) copy online network parameter to target network
  # (3) sample a batch from replay buffer, do the online network update (batch size:50)
  while True:
    step = global_step.eval()
    # if global training steps have meet the maximum, we will stop training
    if step >= n_steps:
      break
    # One iteration is just trakcing ONE sample generation !!!
    iteration+=1
    print("Iteration %d \t Training step %d/%d loss value %f \t Mean Max Q %f" %(iteration, step, n_step, loss_val, mean_max_q))

    if done:
      obs = env.reset()
      for skip in range(skip_start): # skip the beginning of each game
        obs, reward, done, info = env.step(0) # take action 0
      state = preprocess(obs)
    
    # online network evaluates what to do
    q_values = online_q_values.eval(feed_dict={X_state:[state])})
    # q_values returns array of 4 values corresponding to 4 actions
    # for each step in the game, we record the max of q_values 
    action = epsilon_greedy(q_values)

    # online network plays
    obs, reward, done, info = env.step(action)
    next_state = preprocess(obs)

    # Add this sample to the replay buffer
    replay_buffer.append((state, action, reward, next_state, 1.0-done))
    state=next_state

    # compute statistics to track training progress
    total_max_q +=q_values.max()
    game_length +=1
    if done:
      mean_max_q = total_max_q/game_length
      total_max_q=0.0
      game_length = 0
    
    if iteration < training_start or iteration % training_interval !=0:
      continue
    

